# Qiskit Metal Design

In [2]:
# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from collections import OrderedDict

# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

# Packages for the simple design
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# Analysis
# from qiskit_metal.renderers.renderer_gds.gds_renderer import QGDSRenderer
# from qiskit_metal.analyses.quantization import EPRanalysis
from qiskit_metal.analyses.quantization import EPRanalysis
from qiskit_metal.analyses.simulation import ScatteringImpedanceSim
from qiskit_metal.analyses.sweep_and_optimize.sweeping import Sweeping
import pyEPR as epr
import numpy as np

# Set up chip design as multiplanar to use Gmesh/Elmer
design = designs.DesignPlanar({}, overwrite_enabled=True)

# Set up chip dimensions 
design.chips.main.size.size_x = '4.6mm'
design.chips.main.size.size_y = '2.4mm'
design.chips.main.size.size_z = '-280um'
design.chips.main.size.center_x = '0mm'
design.chips.main.size.center_y = '-1mm'


# Resonator and feedline gap width (W) and center conductor width (S) from reference 2
design.variables['cpw_width'] = '10 um' #S from reference 2
design.variables['cpw_gap'] = '6 um' #W from reference 2


# Create GUI
gui = MetalGUI(design)

# Lauchpad 1
x1 = '-2.015mm'
y1 = '0mm'
launch_options1 = dict(chip='main', pos_x=x1, pos_y=y1, orientation='360', lead_length='30um', pad_height='103um', 
                      pad_width='103um', pad_gap='60um')
LP1 = LaunchpadWirebond(design, 'LP1', options = launch_options1)

# Launchpad 2
x2 = '2.015mm'
y1 = '0mm'
launch_options2 = dict(chip='main', pos_x=x2, pos_y=y1, orientation='180', lead_length='30um', pad_height='103um', 
                      pad_width='103um', pad_gap='60um')
LP2 = LaunchpadWirebond(design, 'LP2', options = launch_options2)

# Using path finder to connect the two launchpads
TL = RoutePathfinder(design, 'TL', options = dict(chip='main', trace_width ='10um',
                                            trace_gap ='6um',
                                            fillet='90um',                                                            
                                            hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='LP1',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='LP2',
                                                    pin='tie')
                                            )))

######################
#      Transmon      #
######################

from qiskit_metal.qlibrary.qubits.transmon_pocket_teeth import TransmonPocketTeeth

Q1 = TransmonPocketTeeth(design,'Qubit1', options = dict(
        pos_x='0.5mm',
        pos_y='-1.35mm',
        orientation = '90',
        pad_gap ='30um',                       
        inductor_width    = '20um',                       
        pad_width       = '400um',                      
        pad_height       = '90um',                       
        pocket_width    = '650um',                      
        pocket_height    = '650um',                      
        coupled_pad_height= '30um',                      
        coupled_pad_width = '20um',                       
        coupled_pad_gap   = '50um',
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        connection_pads = dict(
            readout = dict(loc_W=0, loc_H=1, pad_height = '50um', pad_width= '20um', cpw_extend = '20um', pad_cpw_shift='-0', pad_gap ='10um')
            )))

#######################
# lambda/4 resonator1 #
#######################
otg1 = OpenToGround(design, 'otg1', options=dict(chip='main', pos_x='-0.5mm',  pos_y='-40um', orientation = 180))
#stg1 = ShortToGround(design, 'stg1', options=dict(chip='main', pos_x='0mm',  pos_y='-1.35mm', orientation = -90))

# Use RouteMeander to fix the total length of the resonator
res1 = RouteMeander(design, 'resonator1',  Dict(
        trace_width ='10um',
        trace_gap ='6um',
        total_length='3.7mm',
        hfss_wire_bonds = True,
        fillet='99.9 um',
        lead = dict(start_straight='300um'),
        pin_inputs=Dict(
        start_pin=Dict(component='otg1', pin='open'),
        end_pin=Dict(component='Qubit1', pin='readout')), ))

# rebuild the GUI
gui.rebuild()

# SQD Gmsh Renderer

In [3]:
from SQDMetal.PALACE.SQDGmshRenderer import Palace_Gmsh_Renderer
import gmsh

#Clear data from previous model
gmsh.model.remove()
gmsh.finalize()

#Create new Palace Gmsh Renderer object
pgr = Palace_Gmsh_Renderer(design)

#Run this only for eigenmode/RF simluation
pgr.add_ports_on_launchpad(LP1)
pgr.add_ports_on_launchpad(LP2)

#Converts Qiskit Metal design to Gmsh geometries
pgr.convert_design_to_gmsh()

#Print the physical groups/boundary conditions
pgr.print_physical_groups()

#gmsh.model.occ.healShapes(, tolerance=1e-6)
gmsh.model.occ.fuse([(2,5)],[(2,3)])

#Plot design in gmsh gui
pgr.view_design_components()

drawing polygon: LP1
drawing polygon: LP2
drawing path: TL
drawing path: Qubit1
drawing polygon: Qubit1
drawing junction: Qubit1
drawing polygon: otg1
drawing path: resonator1

Physical Groups
name: LP1		 identifier: 1
name: LP2		 identifier: 2
name: TL		 identifier: 3
name: Qubit1		 identifier: 4
name: Qubit1_9		 identifier: 5
name: Qubit1_10		 identifier: 6
name: Qubit1_12		 identifier: 7
name: Qubit1_13		 identifier: 8
name: resonator1		 identifier: 9
name: port_LP1a		 identifier: 10
name: port_LP1b		 identifier: 11
name: port_LP2a		 identifier: 12
name: port_LP2b		 identifier: 13
name: ground_plane		 identifier: 14
name: far_field		 identifier: 17
name: dielectric_base		 identifier: 15
name: air_box		 identifier: 16


Exception: Fuse operation cannot be performed

In [4]:
a

[(2, 43), (3, 3), (2, 39), (2, 40), (2, 41), (2, 42)]

In [3]:
pgr.metal_surfaces

[('LP1', 1),
 ('LP2', 3),
 ('TL', 5),
 ('Qubit1', 7),
 ('Qubit1', 9),
 ('Qubit1', 10),
 ('Qubit1', 12),
 ('Qubit1', 13),
 ('resonator1', 15)]

# Mesh

In [3]:
gmsh.option.setNumber('Mesh.MeshSizeMin',50e-3)
gmsh.option.setNumber('Mesh.MeshSizeMax',120e-3)
gmsh.option.setNumber('Mesh.MshFileVersion', 2.2)
gmsh.model.mesh.generate(dim=3)
gmsh.fltk.run()



# Save Mesh File

In [4]:
gmsh.write("transmon_cap.msh")

In [4]:
gmsh.fltk.run()